In [1]:
import os
import logging, os

logging.disable(logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [4]:
import tensorflow as tf
import keras
from musicnet.preprocessing.wav_specs_and_notes.utils import create_tf_record_ds
from musicnet.models.transformer.Transformer import F1FromSeqLogits, WeightedBinaryCrossentropy, WarmupLRSchedule
from musicnet.utils import load_params, PROJECT_ROOT_DIR, notes_vocab
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
from utils import y_vs_y_pred_vis, spectogram_vis
from ipywidgets import interact
import librosa

In [5]:
model = keras.models.load_model(
    os.path.join(PROJECT_ROOT_DIR, "musicnet", "models", "transformer", "model.keras"),
    {
        "WeightedBinaryCrossentropy": WeightedBinaryCrossentropy,
        "F1FromSeqLogits": F1FromSeqLogits,
        "WarmupLRSchedule": WarmupLRSchedule
    }
)

In [6]:
params = load_params([
    "transformer.*",
    "wav_specs_and_notes.preprocessor.*",
    "wav_specs_and_notes.use_converted_midis"
])

In [7]:
ds_params = {
    "architecture": params["architecture"],
    "n_filters": params["n_filters"],
    "target_classes": model.get_config()["target_classes"],
    "batch_size": params["batch_size"],
    "dataset_size": params["dataset_size"],
    "use_converted_midis": params["use_converted_midis"]
}

In [8]:
train_ds = create_tf_record_ds("train", **ds_params)
val_ds = create_tf_record_ds("val", **ds_params)

In [9]:
model.evaluate(val_ds)

322/322 [==============================] - 18s 47ms/step - loss: 0.7923 - f1_score: 0.1852


[0.7923457622528076, 0.1851751059293747]

In [8]:
for x, y in val_ds:
    break

In [9]:
print(x.shape, y.shape)

(8, 1000, 200) (8, 1000, 83)


In [14]:
y_pred = model(x)
y_pred = tf.sigmoid(y_pred)

In [22]:
spectogram_params = { k: v for k, v in params.items() if k in ["n_fft", "target_sr", "min_hz", "n_filters"] }
spectogram_params

{'target_sr': 44100, 'n_fft': 2205, 'min_hz': 0, 'n_filters': 200}

In [62]:
spectogram_params = { k: v for k, v in params.items() if k in ["n_fft", "target_sr", "min_hz"] }

@interact(i=(0, params["batch_size"] - 1), t=(0.5, 1.0))
def show_sample(i, t=0.5):
    y_vs_y_pred_vis(y[i].numpy(), (y_pred > t)[i].numpy())
    spectogram_vis(x[i].numpy(), **spectogram_params)

interactive(children=(IntSlider(value=3, description='i', max=7), FloatSlider(value=0.5, description='t', max=…

In [217]:
# @tf.function
# def run_inference(context, max_len=999, head_start=0):
#     y_pred = tf.TensorArray(dtype=tf.bool, size=0, dynamic_size=True)
#     for i in tf.range(head_start):
#         y_pred = y_pred.write(i, y[-1, i, :])
#     for i in tf.range(head_start, max_len):
#         next_pred = model([context, [y_pred.stack()]], training=False)[-1, -1, :]
#         next_pred = tf.sigmoid(next_pred)
#         y_pred = y_pred.write(i, next_pred > 0.5)
#     return y_pred.stack()

In [251]:
# y_pred_infer = run_inference(context, y.shape[1], head_start=100)
# y_pred_context = model([context, x])[0]
# y_pred_context = tf.sigmoid(y_pred_context).numpy() > 0.5